In [ ]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 68.1 MB/s eta 0:00:00


In [1]:
# Import libraries:
import numpy as np
from time import time
import gurobipy as gp
from gurobipy import GRB


def solve_model(n_facilities, n_customers, n_scenarios, c, q, K, d):
  # Generate ranges:
  I = range(n_facilities)  # range set for facilities
  J = range(n_customers)  # range set for customers
  S = range(n_scenarios) # range set for scenarios

  # Model
  model = gp.Model("Stochastic Facility Location")
  model.setParam('OutputFlag', 0)  # Disable verbose output

  # Decision variables
  y = model.addVars(I, vtype=GRB.BINARY, name="y")  # Facility open/close
  x = model.addVars(I, J, S, vtype=GRB.CONTINUOUS, name="x")  # Transported quantity

  # Objective function
  model.setObjective(
      gp.quicksum(c[i] * y[i] for i in I) +
      1/n_scenarios * gp.quicksum(q[i, j] * x[i, j, s] for i in I for j in J for s in S),
      GRB.MINIMIZE
  )

  # Constraints
  model.addConstrs(
      (gp.quicksum(x[i, j, s] for i in I) >= d[s, j] for s in S for j in J),
      "Demand"
  )
  model.addConstrs(
      (gp.quicksum(x[i, j, s] for j in J) <= K[i] * y[i] for s in S for i in I),
      "Capacity"
  )

  # Solve
  model.optimize()

  return model.objVal, [y[i].x for i in I]

def solve_second_stage(n_facilities, n_customers, c, q, K, demand, Y_sol):
  # print(n_facilities, n_customers, c, q, K, demand, Y_sol)
  # Generate ranges:
  I = range(n_facilities)  # range set for facilities
  J = range(n_customers)  # range set for customers

  # Model
  model = gp.Model("Facility Location Second Stage")
  model.setParam('OutputFlag', 0)  # Disable verbose output

  # Decision variables
  x = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="x")  # Transported quantity

  # Objective function
  model.setObjective(
      gp.quicksum(q[i, j] * x[i, j] for i in I for j in J),
      GRB.MINIMIZE
  )
  # Constraints
  model.addConstrs(
      (gp.quicksum(x[i, j] for i in I) >= demand[j] for j in J),
      "Demand"
  )
  model.addConstrs(
      (gp.quicksum(x[i, j] for j in J) <= K[i] * Y_sol[i] for i in I),
      "Capacity"
  )

  # Solve
  model.optimize()
  return model.objVal + sum(c[i] * Y_sol[i] for i in I)


def get_scenarios(n_scenarios):
  # Demand per scenario
  return np.random.uniform(
      low=5,
      high=10,
      size = (n_scenarios, n_customers)
    ) + 5 * np.random.uniform(
      low=0,
      high=1,
      size = (n_scenarios, n_customers)
    )

In [2]:
# set seed
np.random.seed(3)
# Generate instance
n_facilities = 6
n_customers = 8
c = [10] * n_facilities   # Facility opening costs
q = np.random.uniform(
    low=5,
    high=10,
    size = (n_facilities, n_customers)
) # Transportation cost
K = [90] * n_facilities  # Facility capacity

In [ ]:
obj_lst = np.ones(10)
for i in range(10):
  n_scenarios = 3
  d = get_scenarios(n_scenarios)
  obj, opt_sol = solve_model(n_facilities, n_customers, n_scenarios, c, q, K, d)
  print(f"{obj:.2f}, {opt_sol}")
  obj_lst[i] = obj
print(f"[{min(obj_lst):.2f},{max(obj_lst):.2f}]")

Restricted license - for non-production use only - expires 2026-11-23
508.85, [1.0, 1.0, 1.0, -0.0, 1.0, 0.0]
504.48, [1.0, 1.0, 1.0, 0.0, 1.0, 0.0]
533.39, [1.0, 1.0, 1.0, 0.0, 1.0, 0.0]
471.85, [1.0, 1.0, 1.0, 0.0, 1.0, 0.0]
533.01, [1.0, 1.0, 1.0, -0.0, 1.0, 0.0]
505.86, [1.0, 1.0, 1.0, -0.0, 1.0, 0.0]
502.23, [0.0, 1.0, 1.0, -0.0, 1.0, 0.0]
512.83, [1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
533.21, [1.0, 1.0, 1.0, -0.0, 1.0, 0.0]
499.50, [1.0, 1.0, 1.0, 0.0, 1.0, -0.0]
[471.85,533.39]


# Do we reach stability?

In [4]:
obj_lst = np.ones(10)
n_scenarios = 20
for i in range(10):
  d = get_scenarios(n_scenarios)
  obj, opt_sol = solve_model(n_facilities, n_customers, n_scenarios, c, q, K, d)
  print(f"{obj:.2f}, {opt_sol}")
  obj_lst[i] = obj
print(f"[{min(obj_lst):.2f},{max(obj_lst):.2f}]")

Restricted license - for non-production use only - expires 2026-11-23


509.36, [1.0, 1.0, 1.0, 0.0, 1.0, -0.0]
510.12, [1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
524.73, [1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
517.40, [1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
500.84, [1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
519.24, [1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
493.20, [1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
508.75, [1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
504.16, [1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
526.54, [1.0, 1.0, 1.0, -0.0, 1.0, 0.0]
[493.20,526.54]


# Do we reach stability?

In [5]:
# Simulate the second stage
N_REPS = 1000
of_lst = np.zeros(N_REPS)
for i in range(N_REPS):
  demand = get_scenarios(1).reshape(n_customers, )
  of = solve_second_stage(n_facilities, n_customers, c, q, K, demand, opt_sol)
  # print(of)
  of_lst[i] = of
print(f"AVG: {np.mean(of_lst):.2f}")

AVG: 510.89


In [6]:
# VSS
# pick average values
avg_d = np.mean(get_scenarios(50), axis=0).reshape(1,n_customers)
start_time = time()
obj_vss, opt_sol_vss = solve_model(n_facilities, n_customers, 1, c, q, K, avg_d)
end_time = time()
print(f"{obj_vss:.2f}, {opt_sol_vss}")
print(f"time EVP: {(end_time-start_time):.2f}")

# Stochastic problem:
d = get_scenarios(n_scenarios)
start_time = time()
obj, opt_sol = solve_model(n_facilities, n_customers, n_scenarios, c, q, K, d)
end_time = time()
print(f"time stochastic: {(end_time-start_time):.2f}")

507.30, [1.0, 1.0, 1.0, -0.0, 1.0, -0.0]
time EVP: 0.00
time stochastic: 0.03


# Is it worth to solve the stochastic problem? why?